# Elevated-Plus Maze Analysis

In [ ]:
from analysis import analysis_utils as au
from IPython.core.interactiveshell import InteractiveShell
from multiprocessing import Process
from multiprocessing import Queue
import numpy as np
import os
import pandas as pd
import random
from scipy import stats
import seaborn as sns
import SigProc
import sys

In [ ]:
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"
sns.set_style("darkgrid")

## Organize all the files for each mouse. Go through the EPM directory, and for each mouse, add the corresponding behavior data file path name and the corresponding Raw data file path name to a dictionary.

In [ ]:
mouse_directory = os.path.expanduser("~") + "/Hen_Lab/Mice/EPM"

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

animal_data_files = dict()

for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    if subdir_list:
        mice_directories = subdir_list

    for file_name in file_list:
        for mouse_name in mice_directories:
            if file_name.endswith(".csv") and (mouse_name in file_name and "behavior" in file_name):
                if mouse_name in animal_data_files:
                    animal_data_files[mouse_name].update({"behavior":dir_name+"/"+file_name})
                else:
                    animal_data_files[mouse_name] = {"behavior":dir_name+"/"+file_name}
            elif file_name.endswith(".csv") and (mouse_name in file_name and "Raw" in file_name):
                if mouse_name in animal_data_files:
                    animal_data_files[mouse_name].update({"Raw":dir_name+"/"+file_name})
                else:
                    animal_data_files[mouse_name] = {"Raw":dir_name+"/"+file_name}

In [ ]:
animal_data_files

## If a behavior data file or a Raw data file is missing for a particular mouse, then display which file is missing. Then proceed to remove all mice from the dictionary that do not have the required data files, i.e., behavior and raw.

In [ ]:
# Check that for each animal a corresponding behavior data file AND Raw neuron data file was found
missing_data = list()
for mouse_name in animal_data_files:
    if not "behavior" in animal_data_files[mouse_name]:
        print("The behavior for {} file is missing".format(mouse_name))
        missing_data.append(mouse_name)
    if not "Raw" in animal_data_files[mouse_name]:
        print("The raw file for {} is missing".format(mouse_name))
        missing_data.append(mouse_name)
    
# Remove all mice from the dictionary if the required data files for them were not found
for animal_name in missing_data:
    if animal_name in animal_data_files:
        del animal_data_files[animal_name]

In [ ]:
animal_data_files

### 1. Generate and store all dataframes for each animal in a dictionary. 
### 2. Plot the correlation heatmap of the neuron concatenated with behavior dataframe for each corresponding mouse
### 3. Plot the cluster map of the cell transients dataframe for each corresponding mouse

In [ ]:
# Store all the dataframes for each animal in a dictionary for quick access, when needed
animal_dataframes = dict()

for animal_name in animal_data_files:
    print(animal_name)
    
    data = pd.read_csv(animal_data_files[animal_name]["Raw"], header=None)
    _, AUC_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(data, 2, 0.5, 0.2, 10)
    
    behavior_column_names = ['Trial_time', 'Recording_time', 'X_center', 'Y_center', 'Area', 'Areachange', 
                         'Elongation', 'Distance_moved', 'Velocity', 'Arena_centerpoint',
                         'Open1_centerpoint', 'Open2_centerpoint',
                         'Closed1_centerpoint', 'Closed2_centerpoint',
                         'OpenArms_centerpoint', 'ClosedArms_centerpoint', 'Result_1']

    behavior_df = pd.read_csv(animal_data_files[animal_name]["behavior"], header=None)
    behavior_df.columns = behavior_column_names
    
    # Only keep every 3rd row of the original behavior dataframe in order to downsample 30 fps --> 10 fps
    behavior_df = au.downsample_dataframe(behavior_df, 3)

    # Define cutoff for what constitutes as a running frame
    VELOCITY_CUTOFF = 4;

    # Adds "Running_frames" column to the end of the behavior dataframe 
    behavior_df["Running_frames"] = np.where(behavior_df["Velocity"] > VELOCITY_CUTOFF, 1, 0)
    
    neuron_concated_behavior = cell_transients_dataframe.join(behavior_df, how="left")
    
    # Add the corresponding AUC, cell transients, and neuron activity concated with behavior dataframes for each animal to the dictionary
    if not animal_name in animal_dataframes:
        animal_dataframes[animal_name] = {"AUC_dataframe": AUC_dataframe, "cell_transients_dataframe": cell_transients_dataframe, "neuron_concated_behavior": neuron_concated_behavior} 
    
    au.plot_correlation_heatmap(neuron_concated_behavior, size=20)
    au.plot_clustermap(cell_transients_dataframe)

## Display the correlation heat map for each animal's corresponding behaviors, e.g., open arms vs. closed arms

In [ ]:
behaviors = ["Arena_centerpoint", "Open1_centerpoint", "Open2_centerpoint", "Closed1_centerpoint", "Closed2_centerpoint", "OpenArms_centerpoint", "ClosedArms_centerpoint"]

for animal in animal_dataframes:
    print(animal)
    for behavior in behaviors:
        print("      ----> {}".format(behavior))
        current_dataframe = animal_dataframes[animal]["neuron_concated_behavior"]
        indices = current_dataframe.loc[current_dataframe[behavior] != 0].index
        au.plot_correlation_heatmap(animal_dataframes[animal]["cell_transients_dataframe"].iloc[indices], size=20)

In [ ]:
def activity_by_neurons(concated_df, neuron_names, *behaviors, frame_rate=10):
    """Computes the neuron activity rates for given behaviors
    
    This function computes the rates for a given animal's activity and  
    neuron, given some set of behaviors.

    Args: 
        concated_df: a concatenated pandas DataFrame of the neuron activity and 
        the corresponding behavior, for a given animal.
        neuron_names: the names of the neurons whose rates are to be computed.
        behaviors: a list of the behaviors for which to compute the activity rates. 
        frame_rate: the framerate to multiply the rate by, default is 10.

    Returns: 
        activity_df: a pandas DataFrame of the neuron activity rates.
    """
    activity_df = pd.DataFrame(columns=behaviors)
    for behavior in behaviors:
        if behavior in concated_df.columns:
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[concated_df[behavior] != 0, neuron_names].mean()
        elif '&' in behavior:
            beh1 = behavior.split('&')[0]
            beh2 = behavior.split('&')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) & ((concated_df[beh2] != 0)), neuron_names].mean()
        elif '|' in behavior:
            beh1 = behavior.split('|')[0]
            beh2 = behavior.split('|')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) | ((concated_df[beh2] != 0)), neuron_names].mean()

    return activity_df

## Create a dictionary that will store the neuron activity rates dataframes for each animal

In [ ]:
neuron_activity_rates = dict()

for animal in animal_dataframes:
    print(animal)
    if not animal in neuron_activity_rates:
        neuron_activity_rates[animal] = activity_by_neurons(animal_dataframes[animal]["neuron_concated_behavior"], animal_dataframes[animal]["cell_transients_dataframe"].columns, "ClosedArms_centerpoint", "OpenArms_centerpoint", "OpenArms_centerpoint&Running_frames", "ClosedArms_centerpoint&Running_frames", "Running_frames")

In [ ]:
import plotly
import plotly.graph_objs as go

def plot_activity_rates(activity_rates_df, plot_title):
    """Wrapper function for Plotly Bar Charts
        
        This function wraps the Plotly library Bar Chart plotting functionality
        in order to quickly and easily create a bar plot for 
    
    Args:
        activity_rates_df: the pandas DataFrame with all the neuron activity means,
        for corresponding behaviors
    """
    x = [beh.replace('_', "<br>") for beh in activity_rates_df.columns]
    y = [activity_rates_df[behavior].mean() for behavior in activity_rates_df.columns]

    trace1 = go.Bar(
        x=x,
        y=y,
        text=y,
        marker=dict(
            color='rgb(255,100,0)',
            line=dict(
                color='rgb(255,0,25)',
                width=2.5),
            ),
        opacity=0.7
    )

    data = [trace1]
    layout = go.Layout(
        title=plot_title,
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

## Plot the neuron activity rates bar plots for each corresponding animal

In [ ]:
for animal in neuron_activity_rates:
    plot_title = "Neuron Activity Rates by Behavior for " + animal
    plot_activity_rates(neuron_activity_rates[animal], plot_title)

## Stack the neuron activity rates dataframes of all the animals, on top of each other

In [ ]:
rates_of_all_neurons = pd.concat(list(neuron_activity_rates.values()), ignore_index=True)
rates_of_all_neurons

## Plot the neuron activity rates of all the neurons, combined

In [ ]:
plot_activity_rates(rates_of_all_neurons, "Neuron Activity Ratres for all Animals Combined")